In [1]:
from datasets import load_dataset
import json
import yaml

/Users/Shared/Public/huggingface/salamandra-2b-instruct/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [21]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


In [15]:
with open('README.md', 'r') as readme:
    lines = readme.readlines()
    
    # Find start and end of the language array in YAML format
    start_index = None
    for i, line in enumerate(lines):
        if line.strip() == "language:":
            start_index = i
            break
    
    end_index = len(lines)
    for j, line in enumerate(lines[start_index+1:], start=start_index+1):
        if not line.startswith('- '):
            end_index = j
            break

    language_section = ''.join(lines[start_index:end_index])
    
    # Load it with PyYAML
    readme_yaml = yaml.safe_load(language_section)
    langs = readme_yaml['language']

print(langs)

['bg', 'ca', 'code', 'cs', 'cy', 'da', 'de', 'el', 'en', 'es', 'et', 'eu', 'fi', 'fr', 'ga', 'gl', 'hr', 'hu', 'it', 'lt', 'lv', 'mt', 'nl', 'nn', '\\no', 'oc', 'pl', 'pt', 'ro', 'ru', 'sh', 'sk', 'sl', 'sr', 'sv', 'uk']


In [22]:
NUM_SAMPLES = 200
SKIP_SAMPLES = 1000
#DATASET_NAME = "oscar-corpus/colossal-oscar-1.0"
DATASET_NAME = "oscar-corpus/oscar"
_set_env("HF_TOKEN")

In [23]:
if "colossal" in DATASET_NAME:
    ds = load_dataset(DATASET_NAME, "default", split="train", streaming=True)
    ds = ds.take(NUM_SAMPLES)

    for ex in ds:
        print(ex["content"])

In [20]:
samples = dict()

for lang in langs:
    if 2 <= len(lang) <= 3:
        try:
            lang = lang.replace("\\", "")
            name = "unshuffled_deduplicated_" + lang

            ds = load_dataset(DATASET_NAME, name, split="train", streaming=True)
            if SKIP_SAMPLES > 0:
                ds = ds.skip(SKIP_SAMPLES)
            ds = ds.take(NUM_SAMPLES)
            samples[lang] = list(ds)
        except ValueError as e:
            print(e)
    else:
        print(f"lang {lang} not in corpus") # oscar not colossus
print(samples["es"])

Using the latest cached version of the module from /Users/macdev/.cache/huggingface/modules/datasets_modules/datasets/oscar-corpus--oscar/84838bd49d2295f62008383b05620571535451d84545037bb94d6f3501651df2 (last modified on Thu Oct 10 21:13:07 2024) since it couldn't be found locally at oscar-corpus/oscar, or remotely on the Hugging Face Hub.


lang code not in corpus
[{'id': 1000, 'text': 'Hola! Se acabaron las ligas y ahora vienen las notas :) Pero vayamos por partes. Ha sido un año complicado porque en la FCF no pude presentar mis retratos (soy más retratero que otra cosa y las categorías en las que podría incluir mis imágenes eran reportaje o foto-street) y me decanté por el tema "Paisaje no urbano". No es un tema que domine si no que estoy en ello. Estudiando a los mejores, trabajándolo y esforzándome mucho, saliendo a las tantas de la madrugada para hacer amaneceres. Es muy exigente, que el día sea lo que buscas (o adaptarse al día!, pero un cielo azul no es opción),...\t\nHola! Ayer estuve fotografiando el partido Italia – All Star Europa en las instalaciones del Sant Boi. Os dejo el reportaje fotográfico.\nHola! Nuevo shooting y nuevas fotos. Esta vez Amazonas. Homenaje a Mad Max III, más allá de la cúpula del trueno. Modelo: Jenni Vázquez. MUA y estilismo de Marina Canals. Shooting realizado en ModStudios de Ricard L

In [24]:
os.makedirs("imatrix/oscar/langs", exist_ok=True)

DATASET_NAME = "imatrix/oscar/calibration-dataset.txt" if SKIP_SAMPLES > 0 else "imatrix/oscar/imatrix-dataset.txt"

# Write all data to a single file "imatrix-dataset.txt" (flattened format with just text and language labels)
with open(DATASET_NAME, "w", encoding="utf-8") as imatrix_file:
    for lang, data in samples.items():
        for item in data:
            # Write the language and text content to the single file
            imatrix_file.write(f"{lang}: {item['text']}\n")

# Write each language's data to "imatrix/lang.txt" in JSON format
for lang, data in samples.items():
    if not (SKIP_SAMPLES > 0):
        with open(f"imatrix/oscar/langs/{lang}.txt", "w", encoding="utf-8") as lang_file:
            # Write each entry as a JSON object in a separate line for easier parsing
            for item in data:
                json.dump(item, lang_file)
                lang_file.write("\n")  # Add a newline for each JSON entry
        

In [13]:
import os
from collections import defaultdict

def count_results_by_language(dataset_dir):
    # Adjust this path to point to your language files directory
    lang_counts = defaultdict(int)
    
    for filename in os.listdir(dataset_dir):
        if filename.endswith(".txt"):  # Assuming language files are in .txt format
            lang_code = filename.split("-")[0]  # Assumes filename starts with lang code
            with open(os.path.join(dataset_dir, filename), "r", encoding="utf-8") as f:
                for line in f:
                    if line.strip():  # Only count non-empty lines
                        lang_counts[lang_code] += 1
    
    total_results = sum(lang_counts.values())
    
    # Print results summary
    print(f"Total Results: {total_results}")
    for lang, count in lang_counts.items():
        print(f"Language {lang}: {count} results")
    
    return lang_counts, total_results

# Example usage
dataset_directory = "./imatrix/oscar/langs"  # Change to your dataset path
count_results_by_language(dataset_directory)


Total Results: 35000
Language el.txt: 1000 results
Language sv.txt: 1000 results
Language pl.txt: 1000 results
Language en.txt: 1000 results
Language ru.txt: 1000 results
Language ga.txt: 1000 results
Language fr.txt: 1000 results
Language sr.txt: 1000 results
Language lv.txt: 1000 results
Language cy.txt: 1000 results
Language lt.txt: 1000 results
Language nn.txt: 1000 results
Language mt.txt: 1000 results
Language no.txt: 1000 results
Language nl.txt: 1000 results
Language uk.txt: 1000 results
Language cs.txt: 1000 results
Language oc.txt: 1000 results
Language ca.txt: 1000 results
Language bg.txt: 1000 results
Language fi.txt: 1000 results
Language da.txt: 1000 results
Language hu.txt: 1000 results
Language sh.txt: 1000 results
Language gl.txt: 1000 results
Language es.txt: 1000 results
Language sk.txt: 1000 results
Language ro.txt: 1000 results
Language et.txt: 1000 results
Language pt.txt: 1000 results
Language hr.txt: 1000 results
Language eu.txt: 1000 results
Language it.txt: 10

(defaultdict(int,
             {'el.txt': 1000,
              'sv.txt': 1000,
              'pl.txt': 1000,
              'en.txt': 1000,
              'ru.txt': 1000,
              'ga.txt': 1000,
              'fr.txt': 1000,
              'sr.txt': 1000,
              'lv.txt': 1000,
              'cy.txt': 1000,
              'lt.txt': 1000,
              'nn.txt': 1000,
              'mt.txt': 1000,
              'no.txt': 1000,
              'nl.txt': 1000,
              'uk.txt': 1000,
              'cs.txt': 1000,
              'oc.txt': 1000,
              'ca.txt': 1000,
              'bg.txt': 1000,
              'fi.txt': 1000,
              'da.txt': 1000,
              'hu.txt': 1000,
              'sh.txt': 1000,
              'gl.txt': 1000,
              'es.txt': 1000,
              'sk.txt': 1000,
              'ro.txt': 1000,
              'et.txt': 1000,
              'pt.txt': 1000,
              'hr.txt': 1000,
              'eu.txt': 1000,
              'it.txt'

In [25]:
import llama_cpp

model = llama_cpp.Llama(model_path="salamandra-2b-instruct_bf16.gguf", n_ctx=512)
input_text = "Hello world"

try:
    tokens = model.tokenize(input_text)
    print(tokens)
except Exception as e:
    print(f"Error: {e}")


ValueError: Model path does not exist: your_model.gguf